In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib 
import matplotlib as mpl 
import seaborn as sns
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics
import string, re
from keras.preprocessing.text import Tokenizer
import string
import scipy
import warnings
import sklearn as sk
import csv
import os
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
from pandas import get_dummies
import matplotlib as mpl 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from wordcloud import WordCloud as wc
from nltk.corpus import stopwords 
import nltk
import json
import sys
import os
import time
from tqdm import tqdm
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

In [3]:
train =  pd.read_csv("/content/drive/My Drive/train.csv")
test  =  pd.read_csv("/content/drive/My Drive/test.csv")
print("Training data shape : ", train.shape)
print("Testing  data shape : ", test.shape)

Training data shape :  (1306122, 3)
Testing  data shape :  (375806, 2)


In [4]:
train = train.sample(frac=0.1, replace=False, random_state=1)


In [5]:
import keras
model=keras.models.load_model("/content/drive/MyDrive/weights-01-0.9566.hdf5")

In [23]:
def load_and_preprocess():
    train = pd.read_csv("/content/drive/MyDrive/train.csv")
    test = pd.read_csv("/content/drive/MyDrive/test.csv")
    print("Test shape : ",test.shape)
    
    train['clean_text'] = train['question_text'].apply(clean_word)
    test['clean_text'] = test['question_text'].apply(clean_word)
    
    ## split to train and val
    train, val = train_test_split(train, test_size=0.08, random_state=2018)


    ## fill up the missing values
    train_X = train["clean_text"].fillna("_##_").values
    val_X = val["clean_text"].fillna("_##_").values
    test_X = test["clean_text"].fillna("_##_").values
    
    ## Tokenize the sentences
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    val_X = tokenizer.texts_to_sequences(val_X)
    test_X = tokenizer.texts_to_sequences(test_X)

    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen)
    val_X = pad_sequences(val_X, maxlen=maxlen)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    ## Get the target values
    train_y = train['target'].values
    val_y = val['target'].values  
    
    #shuffling the data
    np.random.seed(2018)
    trn_idx = np.random.permutation(len(train_X))
    val_idx = np.random.permutation(len(val_X))

    train_X = train_X[trn_idx]
    
    val_X = val_X[val_idx]
    
    train_y = train_y[trn_idx]
    
    val_y = val_y[val_idx]    
    
    return train_X, val_X, test_X, train_y, val_y, tokenizer.word_index

In [24]:
def load_glove_vector(word_index):
    embedding_file = '/content/drive/MyDrive/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): 
      return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(embedding_file))
    all_embedding = np.stack(embeddings_index.values())
    embedding_mean,embedding_std = all_embedding.mean(), all_embedding.std()
    embedding_size = all_embedding.shape[1]
    n_b_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(embedding_mean,embedding_std,(n_b_words,embedding_size))
    for word, i in word_index.items():
        if i >= max_features: 
          continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
          embedding_matrix[i] = embedding_vector       
    return embedding_matrix    

In [6]:
def clean_word(word):
    word = re.sub(r"(\d+)(k)", r"\g<1>000", word)
    word = re.sub(r":", " : ", word)
    word = re.sub(r" e g ", " eg ", word)
    word = re.sub(r" b g ", " bg ", word)
    word = re.sub(r" u s ", " american ", word)
    word = re.sub(r"\0s", "0", word)
    word = re.sub(r",", " ", word)
    word = re.sub(r"\.", " ", word)
    word = re.sub(r"!", " ! ", word)
    word = re.sub(r"\/", " ", word)
    word = re.sub(r"\^", " ^ ", word)
    word = re.sub(r"\+", " + ", word)
    word = re.sub(r"\-", " - ", word)
    word = re.sub(r"\=", " = ", word)
    word = re.sub(r" 9 11 ", "911", word)
    word = word.translate(string.punctuation)
    word = word.lower()
    word = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", word)
    word = re.sub(r"what's", "what is ", word)
    word = re.sub(r"\'s", " ", word)
    word = re.sub(r"\'ve", " have ", word)
    word = re.sub(r"n't", " not ", word)
    word = re.sub(r"i'm", "i am ", word)
    word = re.sub(r"\'re", " are ", word)
    word = re.sub(r"\'d", " would ", word)
    word = re.sub(r"\'ll", " will ", word)
    word = re.sub(r"'", " ", word)
    word = re.sub(r"e - mail", "email", word)
    word = re.sub(r"j k", "jk", word)
    word = re.sub(r"\s{2,}", " ", word)
    word = re.sub('[^a-zA-Z]',' ', word)
    word = re.sub('  +',' ',word)
    return word

In [15]:
embed_size = 300 
max_features = 95000
maxlen = 70 

##final_function_one

In [21]:
def final_function_one(tests): 
    tests["clean_text"]=tests['question_text'].apply(clean_word)
    train =  pd.read_csv("/content/drive/My Drive/train.csv")
    train["clean_text"]=train['question_text'].apply(clean_word)
    train_X = train["clean_text"].fillna("_##_").values
    test_X = tests["clean_text"].fillna("_##_").values
    #print(train_X)
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    test_X = tokenizer.texts_to_sequences(test_X)
    val_x = pad_sequences(test_X, maxlen=maxlen)
    pred_val_y = model.predict([val_x], batch_size=1024, verbose=0)
    #printing the results
    print("-" * 100)
    print(f"Predicted Probabilties for given question of being insincere is/are: ",pred_val_y[0][0])
    if pred_val_y[0][0]>0.5:
      print(f"The class label for given query point is  :","1 i.e asked question is insincere")
    else:
      print(f"The class label for given query point is  :","0 i.e asked question is sincere")    

In [22]:
test_datapoint_func_1 = test.sample(1)
#predicting the output
final_function_one(test_datapoint_func_1)

----------------------------------------------------------------------------------------------------
Predicted Probabilties for given question of being insincere is/are:  0.048834056
The class label for given query point is  : 0 i.e asked question is sincere


In [25]:
%%time
train_X, val_X, test_X, train_y, val_y, word_index = load_and_preprocess()

Test shape :  (375806, 2)
CPU times: user 2min 40s, sys: 2.71 s, total: 2min 43s
Wall time: 2min 51s


##final_function_two

In [35]:
def final_function_two(model, val_y,epochs=1):
    for e in range(epochs):
        pred_val_y = model.predict([val_X], batch_size=1024, verbose=0)
        best_thresh = 0.5
        best_score = 0.0
        for thresh in np.arange(0.1, 0.501, 0.01):
            thresh = np.round(thresh, 2)
            score = metrics.f1_score(val_y, (pred_val_y > thresh).astype(int))
            roc=metrics.roc_auc_score(val_y,pred_val_y)
            if score > best_score:
                best_thresh = thresh
                best_score = score    
        print("Val F1 Score: {:.4f}".format(best_score))
        print("Val ROC Score: {:.4f}".format(roc))
        

In [36]:
final_function_two(model,val_y,1)

Val F1 Score: 0.6509
Val ROC Score: 0.9600
